# 0. Imports

In [ ]:
from my_dash_components import ScatterplotComponent, LinearplotComponent, HistogramplotComponent, LinearHistComponent
from dash import dcc, html, Input, Output, Dash
import pandas as pd

# 1. Gráfico customizado com widget do dash 

In [ ]:
filename =  './web3/data.tsv'
iris = pd.read_csv(filename,sep="\t")
iris_array_of_dicts = iris.to_dict(orient='records')

app = Dash(__name__)

app.layout = html.Div([
    ScatterplotComponent(
        id='input',
        data=iris_array_of_dicts
    ),
    dcc.RangeSlider(
        id='range-slider',
        min=0, max=2.5, step=0.1,
        marks={0: '0', 2.5: '2.5'},
        value=[0.5, 2]
    ),
])

@app.callback(
    Output("input", "data"), 
    Input("range-slider", "value"))
def update_bar_chart(value):
    low, high = value
    mask = (iris['petalWidth'] > low) & (iris['petalWidth'] < high)
    res = iris[mask].to_dict(orient='records')
    return res

if __name__ == '__main__':
    app.run(mode="jupyterlab")


# 2. Pdp com histograma

In [ ]:
path = './data/bike.csv'
df = pd.read_csv(path)
df.head()

In [ ]:
num_features = ['temp', 'hum', 'windspeed']
X = df[num_features]
y = df['cnt']
feature = 0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor(n_estimators=10).fit(X, y)
features, feature_names = [(0,)], [f"Features #{i}" for i in range(X.shape[1])]
deciles = {0: np.linspace(0, 100, num=5)}
pd_results = partial_dependence(
    clf, X, features=feature, kind="average")
display = PartialDependenceDisplay(
    [pd_results], features=features, feature_names=feature_names,
    target_idx=0, deciles=deciles
)
display.plot()

plt.show()

In [ ]:
pd_results.keys()

In [ ]:
features = [0,1,2]
PartialDependenceDisplay.from_estimator(clf, X, features)

In [ ]:
plt.plot(pd_results['grid_values'][0], pd_results['average'][0])
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
sns.histplot(df[num_features[feature]], kde=True)

In [ ]:
pdp_df = pd.DataFrame({'grid_values': pd_results['grid_values'][0], 'average': pd_results['average'][0]})
pdp_dict = pdp_df.to_dict(orient='records')
x_start = min(df[num_features[feature]].min(), pdp_df['grid_values'].min())
x_end = max(df[num_features[feature]].max(), pdp_df['grid_values'].max())

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    LinearplotComponent(
        id='linear',
        data=pdp_dict,
        x_axis='grid_values',
        y_axis='average',
        xStart=x_start,
        xEnd=x_end,
    ),
    HistogramplotComponent(
        id='hist',
        data=df.to_dict(orient='records'),
        value=num_features[feature],
        xStart=x_start,
        xEnd=x_end,
    ),
    html.Div(id='output')
])

clickedValue = ""
@app.callback(
    Output("output", "children"), 
    Input("linear", "value"))
def showDataPoint(value):
    global clickedValue
    clickedValue = value
    return value

if __name__ == '__main__':
    app.run(mode="jupyterlab")

In [ ]:
clickedValue

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    LinearHistComponent(
        id='linear-hist',
        data=pdp_dict,
        histData=df.to_dict(orient='records'),
        x_axis='grid_values',
        y_axis='average',
        histValue=num_features[feature]
    ),
])

if __name__ == '__main__':
    app.run(mode="jupyterlab")



# 3. Tentativa de Encapsulamento

In [ ]:
from my_dash_components.Encapsulation.Encapsulation import Window, ScatterplotGraph, RangeSliderWidget

In [ ]:
def effectOnData(value):
    low, high = value
    mask = (iris['petalWidth'] > low) & (iris['petalWidth'] < high)
    res = iris[mask].to_dict(orient='records')
    return res

app = Window(iris_array_of_dicts)
app.add(ScatterplotGraph()).add(RangeSliderWidget(min=0, max=2.5, step=0.1, effectOnData=effectOnData))
app.run()